# ASPP

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [47]:
# (N, C, H, W)
inplanes = 3
input = torch.rand(2, inplanes, 32, 32)
dilation = [1, 6, 12, 18]


aspp1 = nn.Conv2d(inplanes, 3, kernel_size=1, stride=1, padding=0,
                  dilation=dilation[0], bias=False)(input)
aspp1 = nn.BatchNorm2d(3)(aspp1)
aspp1 = nn.ReLU()(aspp1)

aspp2 = nn.Conv2d(inplanes, 3, kernel_size=3, stride=1, padding=dilation[1],
                  dilation=dilation[1], bias=False)(input)
aspp2 = nn.BatchNorm2d(3)(aspp2)
aspp2 = nn.ReLU()(aspp2)

aspp3 = nn.Conv2d(inplanes, 3, kernel_size=3, stride=1, padding=dilation[2],
                  dilation=dilation[2], bias=False)(input)
aspp3 = nn.BatchNorm2d(3)(aspp3)
aspp3 = nn.ReLU()(aspp3)

aspp4 = nn.Conv2d(inplanes, 3, kernel_size=3, stride=1, padding=dilation[3],
                  dilation=dilation[3], bias=False)(input)
aspp4 = nn.BatchNorm2d(3)(aspp4)
aspp4 = nn.ReLU()(aspp4)

global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                               nn.Conv2d(inplanes, 3, 1, stride=1, bias=False),
                               nn.BatchNorm2d(3),
                               nn.ReLU())(input)
print("dilation={}, Output shape: {}".format(dilation[0], aspp1.shape),
      "dilation={}, Output shape: {}".format(dilation[1], aspp1.shape),
      "dilation={}, Output shape: {}".format(dilation[2], aspp1.shape),
      "dilation={}, Output shape: {}".format(dilation[3], aspp1.shape),
      "global_avg_pool, Output shape: {}".format(global_avg_pool.shape), sep="\n\n")

global_avg_pool = F.interpolate(global_avg_pool,
                                size=aspp4.size()[2:],
                                mode='bilinear',
                                align_corners=True)

print("\nglobal_avg_pool.upsample, Output shape: {}".format(global_avg_pool.shape))

x = torch.cat((aspp1, aspp2, aspp3, aspp4, global_avg_pool), dim=1)
print("\nConcatenate: ", x.shape)

out = nn.Conv2d(15, 3, 1, bias=False)(x)
out = nn.BatchNorm2d(3)(out)
out = nn.ReLU()(out)

print("\nOutput: ", out.shape)

dilation=1, Output shape: torch.Size([2, 3, 32, 32])

dilation=6, Output shape: torch.Size([2, 3, 32, 32])

dilation=12, Output shape: torch.Size([2, 3, 32, 32])

dilation=18, Output shape: torch.Size([2, 3, 32, 32])

global_avg_pool, Output shape: torch.Size([2, 3, 1, 1])

global_avg_pool.upsample, Output shape: torch.Size([2, 3, 32, 32])

Concatenate:  torch.Size([2, 15, 32, 32])

Output:  torch.Size([2, 3, 32, 32])
